In [3]:
from mysike import SIKE_compute_pk, eA, eB, p, PA, QA, PB, QB, A
from numpy import random
from client import app
from classes import db, OneTimeKey


c:\Users\quent\Desktop\Université\Thèse\client\env\lib\site-packages\flask_sqlalchemy\__init__.py:833: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [6]:
%timeit -n 100 for i in range(100) : SIKE_compute_pk(random.randint(1, 3**eB), 'receiver')

739 ms ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
from time import sleep
import requests
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
# Calcul des paires OTK, privées et publiques
def generate_otk(name, method):
    j = 1

    response = requests.get("http://127.0.0.1:5000/onetimekey")
    last_index = int(response.text)
    all_otks = []
    for i in range(100):
        
        if method=="sike27":
            private_key = random.randint(1, min(2**eA, 3**eB))
            public_key = SIKE_compute_pk(private_key, "receiver")
        elif method=="ecdh":
            private_key = X25519PrivateKey.generate()
            public_key = X25519PrivateKey.generate().public_key()
        elif method=="SIKE751":
            private_key = random.randint(1, min(2**eA, 3**eB))
            public_key = 12345678
            
        otk = {'owner': name, 'publicOTK': repr(public_key)}
        all_otks += [otk]

        j += 1
    if method=="SIKE751":
        sleep(0.487)

    requests.post("http://127.0.0.1:5000/onetimekey", json={'keys' : all_otks})

    return "Done"

%timeit -n 10 generate_otk(str(random.randint(10000)), "sike27")
%timeit -n 10 generate_otk(str(random.randint(10000)), "ecdh")
%timeit -n 10 generate_otk(str(random.randint(10000)), "SIKE751")

3.08 s ± 95.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.14 s ± 167 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.26 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
from classes import OneTimeKey
import requests
from time import time
from numpy import random

t1 = time()
data =  {'name': str(random.randint(1000)), 'members': ['Quentin', 'Bob', 'Charlie']}
response = requests.post('http://127.0.0.1:8080/create_group', json=data)
t2 = time()
print('Time :', t2-t1)

Time : 0.10194206237792969


In [8]:
from mysike import compute_pk
number = random.randint(1, 3**eB)
%timeit compute_pk(number, 'receiver', method='sike27')

%timeit for _ in range(100): compute_pk(random.randint(1, 3**eB), 'receiver', method='sike27')


7.46 ms ± 497 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
776 ms ± 43.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
